![Top <](./images/watsonxdata.png "watsonxdata")

# Lab 3: Filtered Queries

In the previous lab searches were only performed on the vector field. In this lab the nearest neighbor search on the vector field is combined with additional search conditions on scalar fields.

The first steps for creating and loading a database are the same as in lab 1 and 2. If you just have executed lab 1 you can quickly execute the following steps until you reach the section **Filtered Querying of Milvus**. 

The first step is to make sure that the Milvus extensions are loaded into the notebook. 

In [ ]:
!pip install pymilvus==2.6.2

## Local Connection

A local connection assumes that you are running your Jupyter notebook inside the same server that is running watsonx.data and the Milvus server. The connection user is the default watsonx.data userid (ibmlhadmin). You need to generate the certificate that will be used by the connection.

### Generate the Connection Certificate

In [ ]:
!rm -f /tmp/presto.cert
!echo QUIT | openssl s_client -showcerts -connect localhost:8443 | awk '/-----BEGIN CERTIFICATE-----/ {p=1}; p; /-----END CERTIFICATE-----/ {p=0}' > /tmp/presto.crt

In [ ]:
rc = %system echo QUIT | openssl s_client -showcerts -connect watsonxdata:8443 | \
        awk '/-----BEGIN CERTIFICATE-----/ {p=1}; p; /-----END CERTIFICATE-----/ {p=0}' > /tmp/presto.crt 

### Local Connection Parameters

In [ ]:
host            = 'watsonxdata'
port            = 19530
apiuser         = 'xxxxxxxxxx'
apikey          = 'xxxxxxxx'
server_pem_path = '/tmp/presto.crt'

## Milvus Connection

In [ ]:
from pymilvus import(
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

connections.connect(alias='default',
                   host=host,
                   port=port,
                   user=apiuser,
                   password=apikey,
                   server_pem_path=server_pem_path,
                   server_name='watsonxdata',
                   secure=True)

### Check Connection Status

In [ ]:
print(f"\nList connections:")
print(connections.list_connections())

## Create a Collection in Milvus
This code will drop the wiki_articles collection if it exists, and then recreate it. This script should return the following text.
```
Status(code=0, message=)
```

#### Make various unitilty commands available

In [ ]:
from pymilvus import utility

#### Clean up previous collection if one already exists

In [ ]:
utility.drop_collection("wiki_articles")

#### Create a sample collection

In [ ]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True), # Primary key
    FieldSchema(name="article_text", dtype=DataType.VARCHAR, max_length=2500,),
    FieldSchema(name="article_title", dtype=DataType.VARCHAR, max_length=200,),
    FieldSchema(name="article_subtopic", dtype=DataType.VARCHAR, max_length=10,),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=384),
]

schema = CollectionSchema(fields, "wikipedia article collection schema")

wiki_collection = Collection("wiki_articles", schema)

#### Create an index for this collection

- metric_type specifies the distance metric used in the vector space. L2 is the Euclidian distance.
- index_type specifies the type of vector index to use. IVF means inverted file index which means clusting the the vector space and representing each cluster by its centroid. FLAT means that vectors are stored directly without any compression or quantization meaning that precise distance calculations are possible
- params specifies several parameters relevant for our index. For instance nlist defines the number clusters to use for the inverted file index. 

In [ ]:
index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{"nlist":2048}
}

wiki_collection.create_index(field_name="vector", index_params=index_params)

#### Double Check that the Schema Exists

In [ ]:
from pymilvus import utility
utility.list_collections()

## Get data from Wikipedia for loading into our collection

In [ ]:
import wikipedia
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# search
search_results = wikipedia.search("Climate")

articles = []
for i in range (0,len(search_results)):
    try:
        summary = wikipedia.summary(search_results[i],auto_suggest=False)
    except Exception as err:
        print(f"Skipped article '{search_results[i]}' skipped because of ambiguity.")
        continue
    try:
        page = wikipedia.page(search_results[i],auto_suggest=False).content
    except Exception as err:
        print(f"Skipped article '{search_results[i]}' skipped because of ambiguity.")
        continue

    
    articles.append({
        "title"   : search_results[i],
        "summary" : summary,
        "page"    : page
    })

#print(display_articles)

df = pd.DataFrame.from_dict(articles)
df.style.set_properties(**{'text-align': 'left'})
print(df)

In [ ]:
print(articles)

## Split Articles into chunks

### Define function for splitting article into chunks

In [ ]:
# Chunk data
def split_into_chunks(text, chunk_size):
    words = text.split()
    #print('text:',text)
    #print('words:',words)
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

### Create list of chunks for all articles and create analog list for additional metadata correspong to the chunk (title, subtopic)

In [ ]:
chunk_size=255
passages=[]
passages_titles=[]
passages_subtopic=[]

for a in articles:
    print('title',a['title'])
    if a['title'] == "Climate":
        subtopic="false"
    else:
        subtopic="true"

    p = a['page']
    cl = split_into_chunks(p,chunk_size)

    print("number of chunks=",len(cl))
    for c in cl:
        passages.append(c)
        passages_titles.append(a['title'])
        passages_subtopic.append(subtopic)

### Create the embeddings for the chunks

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
passages_embeddings = model.encode(passages)


### Insert all data into the collection created above

In [ ]:
basic_collection = Collection("wiki_articles") 
data = [
    passages,
    passages_titles,
    passages_subtopic,
    passages_embeddings
]
out = basic_collection.insert(data)
basic_collection.flush()  # Ensures data persistence
print("Done")

## Filtered Querying of Milvus 

The following code shows how you can query by specifying a scaler condition in addition to the vector search

### Load the Collection into memory and check that the Collection has been Loaded

In [ ]:
basic_collection = Collection("wiki_articles") 
basic_collection.load()
basic_collection.num_entities 

## Query Milvus & Prompt LLM
After gathering the data from Wikipedia and then vectorizing it and inserting into Milvus, we are now ready to perform queries against the vector database. We will use the `sentence-transformers/all-MiniLM-L6-v2` model to generate the query vector and then use Milvus to find the most similar vectors in the database.

### Create a Query Function
The following function will be used to query the Milvus database with filtering. The main difference to lab 2 is that now additional parameter search_expression is passed to query_milvus which specifies the additional scalar filter expression.

In [ ]:
from sentence_transformers import SentenceTransformer
from pymilvus import(
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

def query_milvus(query, search_expression, num_results=5):
    
    # Vectorize query
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
    query_embeddings = model.encode([query])

    # Search
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 5}
    }
    results = basic_collection.search(
        data=query_embeddings, 
        anns_field="vector", 
        param=search_params,
        limit=num_results,
        expr=search_expression, 
        output_fields=['article_text','article_subtopic'],
    )
    return results

### Prompt LLM with Query Results
Consider how climate change may relate to other industries and processes related to your business. Select one of the questions below to feed into Milvus query.

In [ ]:
question_text = "What can my company do to help fight climate change?"
#question_text = "How do businesses negatively effect climate change?"
#question_text = "What can a businesses do to have a positive effect on climate change?"
#question_text = "How can a business reduce their carbon footprint?"

### Search a Question in Milvus

We want to use the above question_text to perform a approximate nearest neighbor search in Milvus. But in addition we want to filter on the scalar column article_subtopic. We want only results where article_subtopic is "false". We search for the most relevant chunks with these two conditions. The top 3 related chunks are retrieved below and can be used for a large language prompt.



In [ ]:
num_results = 3
filter='article_subtopic == "false"'
results = query_milvus(question_text, filter, num_results)

## Display result

The documents that best match the question are now displayed in the list below.

In [ ]:
import re
display_articles = []
relevant_chunks  = []
for i in range(num_results):
    display_articles.append({
        "ID"      : results[0].ids[i],
        "Distance": results[0].distances[i],
        "Subtopic": results[0][i].entity.get('article_subtopic'),
        # "Article" : re.sub(r"^.*?\. (.*$)",r"\1",results[0][i].entity.get('article_text'))
        "Article" : re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text'))        
    })
    relevant_chunks.append(re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text')))

df = pd.DataFrame.from_dict(display_articles).sort_values("Distance",ascending=False)
df.style.set_properties(**{'text-align': 'left'}).set_caption(question_text).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '20px')
    ]
}])

### Search a Question in Milvus again

We want to repeat the above query but now we want chunks where article_subtopic is "true".

In [ ]:
num_results = 3
filter='article_subtopic == "true"'
results = query_milvus(question_text, filter, num_results)

## Display result

The documents that best match the question are now displayed in the list below.

In [ ]:
import re
display_articles = []
relevant_chunks  = []
for i in range(num_results):
    display_articles.append({
        "ID"      : results[0].ids[i],
        "Distance": results[0].distances[i],
        "Subtopic": results[0][i].entity.get('article_subtopic'),
        # "Article" : re.sub(r"^.*?\. (.*$)",r"\1",results[0][i].entity.get('article_text'))
        "Article" : re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text'))        
    })
    relevant_chunks.append(re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text')))

df = pd.DataFrame.from_dict(display_articles).sort_values("Distance",ascending=False)
df.style.set_properties(**{'text-align': 'left'}).set_caption(question_text).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '20px')
    ]
}])

The distance of the chunks from the query_text are also very different depending on the scalar condition

### Let us check the query result without an additional scalar condition

We want to repeat the above query but now we want chunks without restricting on an additional scaler condition

In [ ]:
num_results = 3
filter=None
results = query_milvus(question_text, filter, num_results)

## Display result

The documents that best match the question are now displayed in the list below.

In [ ]:
import re
display_articles = []
relevant_chunks  = []
for i in range(num_results):
    display_articles.append({
        "ID"      : results[0].ids[i],
        "Distance": results[0].distances[i],
        "Subtopic": results[0][i].entity.get('article_subtopic'),
        # "Article" : re.sub(r"^.*?\. (.*$)",r"\1",results[0][i].entity.get('article_text'))
        "Article" : re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text'))        
    })
    relevant_chunks.append(re.sub(r"^.*?\. (.*\.).*$",r"\1",results[0][i].entity.get('article_text')))

df = pd.DataFrame.from_dict(display_articles).sort_values("Distance",ascending=False)
df.style.set_properties(**{'text-align': 'left'}).set_caption(question_text).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '20px')
    ]
}])

#### Credits: IBM 2025, Wilfried Hoge [hoge@de.ibm.com] and Andreas Weininger [andreas.weininger@de.ibm.com] based on a notebook by George Baklarz [baklarz@ca.ibm.com]